**CARDINALITY INFO**

In [ ]:
def DataCardinalityInfo(df):
    data = df
    print(data.shape)
    cols = set();useless_cols = set()
    cols = data.columns
    
    for i in cols:
      if data.shape[0]-10 < data[i].nunique():
        useless_cols.add(i)
    final_cols = set(cols)-set(useless_cols)
    
    list_col = [];list_unique =[];list_dtype = []
    for i in final_cols:
      list_col.append(i); list_dtype.append(data[i].dtype); list_unique.append(data[i].nunique())
    
    info_data = pd.concat([pd.DataFrame(list_col), pd.DataFrame(list_dtype), pd.DataFrame(list_unique)], axis = 1)
    info_data.columns = ['col_name', 'dtype', 'unique_values']
    print('\nInfo From Cardinality Data:')
    print(info_data)
    
    info = info_data.groupby('dtype', sort = 'False')['unique_values'].mean()
    print('\nmean info from cardinality info:');print(info)
    
    obj_unique_mean = None; int_unique_mean=None
    for i in range(len(info)):  
      if info.index[i] == np.dtype(np.object):
        obj_unique_mean = info[i]
      if info.index[i] == np.dtype(np.int):
        int_unique_mean = info[i]
    
    return obj_unique_mean, int_unique_mean

**FEATURES SEGREGATION**

In [1]:
def Segregation(df, target):
                                                                                
    # Converting it into pandas dataframe
    data = pd.DataFrame(df)
    
    # Dropping Target Variable Column
    if target in data.columns:
      data = data.drop(target, axis = 1)
    else:
      print('\n there is No target column moving forward')
    
    # Creating Containers for numerical columns and categorial columns
    num_cols = set(data._get_numeric_data().columns)                       # Numerical Columns
    non_num_cols = set(data.columns)-set(num_cols)                         # Categorial Columns(or non-num)
    
    print('\nTotal num-cols: {}'.format(len(num_cols)))
    print('\nTotal non-num-cols: {}'.format(len(non_num_cols)))
    
    # container for completely non-useful columns (will help for analysis)
    useless_cols = set()
    
    ############# PART FOR GETTING USEFUL COLS FROM CATEGORIAL COLS  ################  

    # Getting mean of unique values
    obj_unique_mean, int_unique_mean = DataCardinalityInfo(data)
    
    if len(non_num_cols) == 0:
      print('All cols in the data are numerical')
    else:
      # Container for useful obj columns
      useful_obj_cols = set()
      
      # Categorial columns having unique values less than no_of_rows*max_card_factor will be considered as useful 
      if obj_unique_mean:
        if obj_unique_mean<5:
          obj_unique_mean = 5   # Because we can afford 10 dummy columns in final dataframe
        
        for i in non_num_cols:
          if data[i].nunique() < obj_unique_mean:
            useful_obj_cols.add(i)
          else:
            useless_cols.add(i)
    
      print('\nTotal useful cols(having unique values less than {}) found among categorial cols(or non-num-cols): {}' .format(obj_unique_mean, len(useful_obj_cols)))
    
    # Removing Columns like Id
    full_cardinality_relief = 10
    
    full_card = set()
    for i in num_cols:
      if data.shape[0]-full_cardinality_relief < data[i].nunique() and df[i].dtype != np.dtype(np.float) :
        full_card.add(i)
    
    # Removing useless_cols from num_cols container
    num_cols = set(num_cols)-set(full_card)
    
    useless_cols.update(full_card); del full_card
    if len(useless_cols) == 0:
      print('\nNo useless columns dropped')
    else:
      print('\nThere is/are {} columns which are useless. These include:\n {}' .format(len(useless_cols), useless_cols))
    
    
    
    ############ PART FOR RETRIEVING  DISCRETE COLS from NUMERICAL COLS ##################

    # Its important to create a set for discrete cols because only these will be converted to dummy variables
    discrete_cols = set()

    # this factor decides how many unique values are required in order to be a discrete column
    if int_unique_mean:
      if int_unique_mean>200:
        int_unique_mean = 200
      for i in num_cols:
        if data[i].nunique() < int_unique_mean: 
          discrete_cols.add(i)   
    num_cols = set(num_cols) - set(discrete_cols)
    
    # Joining useful obj cols with discrete cols retrieved here    
    discrete_cols.update(useful_obj_cols)
    print('\nFinal discrete cols found that will be converted to dummy variables : {} which are:\n{}'.format(len(discrete_cols), discrete_cols))  
    
    return num_cols, discrete_cols, useless_cols